In [1]:
using Distributions, LinearAlgebra, DifferentialEquations, DataFrames,Colors, PlotlyJS, CSV, ProgressBars
include("../../equation.jl")
using .equation
include("../../LSMC/lsmc_bsde.jl")

WebIO._IJuliaInit()

LSMC (generic function with 1 method)

In [2]:
T=1.0; r=0.01; S0 = 100.0; σ = 0.2; K = 100 # abs_sup_0724
R = 0.3; μ = 0.03;

drift(x) = μ*x
diffusion(x) = σ*x
driver(t, x, y, z) = (
        -r*max(y-z/σ, 0.0)
        -R*min(y-z/σ, 0.0)
        -μ*z/σ
        )
terminal(x) = max(x-K,0.0)
_bsde = BSDE(T, S0, drift, diffusion, driver, terminal);

In [3]:
#### LSMC
#### Laguerre polynomials
function Laguerre1D(x,k)
    if k == 0
        return 1.0
    elseif k == 1
        return (1.0-x)
    elseif k == 2
        return (x^2-4*x+2)/2
    elseif k == 3
        return (-x^3+9*x^2-18*x+6)/6
    elseif k == 4
        return (x^4-16*x^3+72*x^2-96*x+24)/24
    elseif k == 5
        return (-x^5+25*x^4-200*x^3+600*x^2-600*x+120)/120
    elseif k == 6
        return (x^6-36*x^5+450*x^4-2400*x^3+5400*x^2-4320*x+720)/720
    elseif k == 7
        return (-x^7+49*x^6-882*x^5+7350*x^4-29400*x^3+52920*x^2-35280*x+5040)/5040
    elseif k == 8
        return (x^8-64*x^7+1568*x^6-18816*x^5+117600*x^4-376320*x^3+564480*x^2-322560*x+40320)/40320
    elseif k == 9
        return (-x^9+81*x^8-2592*x^7+42336*x^6-381024*x^5+1905120*x^4-5080320*x^3+6531840*x^2-3265920*x+362880)/362880
    else
        return 0.0
    end
end

function bases1D(x, poly)
    vec = zeros(poly+1)
    for k = 0:poly
        vec[k+1] = Laguerre1D(x, k)
    end
    return vec
end

bases1D (generic function with 1 method)

In [4]:
df = DataFrame([[],[],[],[],[],[],[]], ["M", "N", "poly", "samples", "mean", "std", "elapsed_time"])
for M in [2^22]
    for N in [4,5,6,7,8,9]
        poly = N
        function ba(x)
            return bases1D(x, poly)
        end
        #### Generate 50 independent samples
        samples = 50;
        Y0 = zeros(samples);
        exc_start = time()
        for k in ProgressBar(1:samples)
            Y0[k] = LSMC1D(_bsde, ba, M, N);
        end
        exc_stop = time()
        elapsed_time = exc_stop-exc_start
        print(string("M=",M,"_N=",N,"_poly=",poly,":LSMCY_0(",S0,")=",mean(Y0),"/std=",std(Y0),". (",elapsed_time," seconds)\n"))
        print("\n")
        row=[M,N,poly,samples,mean(Y0),std(Y0),elapsed_time]    
        push!(df,row)
    end
end

df |> CSV.write(string("lsmc20231108.csv"))

0.0%┣                                              ┫ 0/50 [00:01<00:-49, -1s/it]
2.0%┣▉                                          ┫ 1/50 [00:21<Inf:Inf, InfGs/it]
4.0%┣█▉                                             ┫ 2/50 [00:41<32:35, 41s/it]
6.0%┣██▉                                            ┫ 3/50 [01:00<23:39, 30s/it]
8.0%┣███▊                                           ┫ 4/50 [01:20<20:25, 27s/it]
10.0%┣████▋                                         ┫ 5/50 [01:40<18:44, 25s/it]
12.0%┣█████▌                                        ┫ 6/50 [02:00<17:35, 24s/it]
14.0%┣██████▍                                       ┫ 7/50 [02:20<16:41, 23s/it]
16.0%┣███████▍                                      ┫ 8/50 [02:40<15:57, 23s/it]
18.0%┣████████▎                                     ┫ 9/50 [02:59<15:19, 22s/it]
20.0%┣█████████                                    ┫ 10/50 [03:19<14:45, 22s/it]
22.0%┣██████████                                   ┫ 11/50 [03:39<14:14, 22s/it]
24.0%┣██████████▉           

M=4194304_N=4_poly=4:LSMCY_0(100.0)=25.498932839818664/std=0.03249017408225531. (983.471321105957 seconds)



100.0%┣████████████████████████████████████████████┫ 50/50 [16:23<00:00, 20s/it]
100.0%┣████████████████████████████████████████████┫ 50/50 [16:23<00:00, 20s/it]
0.0%┣                                               ┫ 0/50 [00:00<00:00, -0s/it]
2.0%┣▉                                          ┫ 1/50 [00:24<Inf:Inf, InfGs/it]
4.0%┣█▉                                             ┫ 2/50 [00:48<38:27, 48s/it]
6.0%┣██▉                                            ┫ 3/50 [01:12<28:07, 36s/it]
8.0%┣███▊                                           ┫ 4/50 [01:37<24:43, 32s/it]
10.0%┣████▋                                         ┫ 5/50 [02:01<22:42, 30s/it]
12.0%┣█████▌                                        ┫ 6/50 [02:26<21:22, 29s/it]
14.0%┣██████▍                                       ┫ 7/50 [02:53<20:42, 29s/it]
16.0%┣███████▍                                      ┫ 8/50 [03:17<19:43, 28s/it]
18.0%┣████████▎                                     ┫ 9/50 [03:41<18:55, 28s/it]
20.0%┣█████████             

M=4194304_N=5_poly=5:LSMCY_0(100.0)=25.937787290767854/std=0.0396825600754363. (1204.202376127243 seconds)



100.0%┣████████████████████████████████████████████┫ 50/50 [20:04<00:00, 25s/it]
100.0%┣████████████████████████████████████████████┫ 50/50 [20:04<00:00, 25s/it]
0.0%┣                                               ┫ 0/50 [00:00<00:00, -0s/it]
2.0%┣▉                                          ┫ 1/50 [00:28<Inf:Inf, InfGs/it]
4.0%┣█▉                                             ┫ 2/50 [00:57<45:37, 57s/it]
6.0%┣██▉                                            ┫ 3/50 [01:29<34:51, 44s/it]
8.0%┣███▊                                           ┫ 4/50 [01:58<30:04, 39s/it]
10.0%┣████▋                                         ┫ 5/50 [02:27<27:29, 37s/it]
12.0%┣█████▌                                        ┫ 6/50 [02:55<25:41, 35s/it]
14.0%┣██████▍                                       ┫ 7/50 [03:24<24:21, 34s/it]
16.0%┣███████▍                                      ┫ 8/50 [03:52<23:15, 33s/it]
18.0%┣████████▎                                     ┫ 9/50 [04:21<22:18, 33s/it]
20.0%┣█████████             

...

M=4194304_N=8_poly=8:LSMCY_0(100.0)=26.30012674677225/std=0.06105104319064669. (2021.936784029007 seconds)



100.0%┣████████████████████████████████████████████┫ 50/50 [33:42<00:00, 41s/it]
100.0%┣████████████████████████████████████████████┫ 50/50 [33:42<00:00, 41s/it]
0.0%┣                                               ┫ 0/50 [00:00<00:00, -0s/it]
2.0%┣▉                                          ┫ 1/50 [00:47<Inf:Inf, InfGs/it]
4.0%┣█▊                                          ┫ 2/50 [01:33<01:14:32, 93s/it]
6.0%┣██▉                                            ┫ 3/50 [02:20<54:41, 70s/it]
8.0%┣███▊                                           ┫ 4/50 [03:06<47:33, 62s/it]
10.0%┣████▋                                         ┫ 5/50 [03:53<43:37, 58s/it]
12.0%┣█████▌                                        ┫ 6/50 [04:39<40:55, 56s/it]
14.0%┣██████▍                                       ┫ 7/50 [05:25<38:52, 54s/it]
16.0%┣███████▍                                      ┫ 8/50 [06:12<37:10, 53s/it]
18.0%┣████████▎                                     ┫ 9/50 [06:58<35:43, 52s/it]
20.0%┣█████████             

M=4194304_N=9_poly=9:LSMCY_0(100.0)=26.340413582845287/std=0.05798417921933401. (2340.4301409721375 seconds)



100.0%┣████████████████████████████████████████████┫ 50/50 [39:00<00:00, 48s/it]
100.0%┣████████████████████████████████████████████┫ 50/50 [39:00<00:00, 48s/it]


"lsmc20231108.csv"